In [20]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import random

In [90]:
# Заголовки, чтобы сделать запрос более "человеческим"
headers = {
       "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
       "Accept-Language": "en-US,en;q=0.5",
       "Referer": "https://google.ru/",
   }

# base_url = "https://www.tripadvisor.ru/Attractions-g293974-Activities-c47-t17-Istanbul.html"
# base_url = 'https://www.tripadvisor.ru/Attractions-g293974-Activities-c26-t142-Istanbul.html'
# base_url = 'https://www.tripadvisor.ru/Attractions-g293974-Activities-c26-t143-Istanbul.html'
# base_url = 'https://www.tripadvisor.ru/Attractions-g293974-Activities-c57-t70-Istanbul.html'
# base_url = 'https://www.tripadvisor.ru/Attractions-g293974-Activities-c52-t98-Istanbul.html'
# base_url = 'https://www.tripadvisor.ru/Attractions-g293974-Activities-c57-t119-Istanbul.html'
# base_url = 'https://www.tripadvisor.ru/Attractions-g293974-Activities-c47-t34-Istanbul.html'
# base_url = 'https://www.tripadvisor.ru/Attractions-g293974-Activities-c47-t163-Istanbul.html'
# base_url = 'https://www.tripadvisor.ru/Attractions-g293974-Activities-c49-t161-Istanbul.html'
base_url = 'https://www.tripadvisor.ru/Attractions-g293974-Activities-c61-t134-Istanbul.html'

In [10]:
# response2 = requests.get('https://www.tripadvisor.ru/Attraction_Review-g293974-d19852123-Reviews-Narmanli_Han-Istanbul.html', headers=headers)
# soup2 = BeautifulSoup(response2.text, 'lxml')


In [11]:
# hotels_name = soup2.find('div', class_='f k O Cj Pj PN Pw PA').find_all('span', class_='biGQs _P pZUbB avBIb osNWb')[-1].text.strip()
# hotels_name

'Narmanli Han'

In [91]:
def scrape_hotel_info(url):
    response1 = requests.get(url, headers=headers)
    if response1.status_code != 200:
        print(f"Ошибка: {response1.status_code}")
        return
    
    soup1 = BeautifulSoup(response1.text, 'lxml')
    dict_hotels = {}
    
    try:
        adress = soup1.find('div', class_='ZhNYD').find('span', class_='biGQs _P XWJSj Wb').text.strip()
        dict_hotels['Адрес'] = adress
    except:
        dict_hotels['Адрес'] = None
        
    # Находим элементы, содержащие информацию об отелях
    hotels_name = soup1.find('div', class_='f k O Cj Pj PN Pw PA').find_all('span', class_='biGQs _P pZUbB avBIb osNWb')[-1].text.replace('\xa0', ' ').strip()
    dict_hotels['Название достопримечательности'] = hotels_name
    
    try:
        marks = soup1.find('div', class_='bdeBj e').find('div', class_='biGQs _P fiohW hzzSG uuBRH').text.strip()
        dict_hotels['Общая оценка'] = marks
    except:
        dict_hotels['Общая оценка'] = None
    
    try:
        reviews = soup1.find('span', class_='biGQs _P pZUbB KxBGd').text.replace('\xa0','').strip()
        dict_hotels['Кол-во отзывов'] = reviews
    except:
        dict_hotels['Кол-во отзывов'] = None
    
    try:
        type_of_dostoprim = soup1.find('span', class_='eojVo').text.strip()
        dict_hotels['Тип достопримечательности'] = type_of_dostoprim
    except:
        dict_hotels['Тип достопримечательности'] = None

    print(dict_hotels)
    return dict_hotels    

In [92]:
# Функция для извлечения информации о отелях
def scrape_hotels(url):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Ошибка: {response.status_code}")
        return
    
    soup = BeautifulSoup(response.text, 'lxml')
    
    # Находим элементы, содержащие информацию об отелях
    hotels = soup.find('div', class_='XmUgu _T Ft MJ').find_all('section', {'class':'mowmC', 'data-automation':'WebPresentation_SingleFlexCardSection'})
    # print('hotels from def', hotels)
    
    counter = 0
    dostoprim_list = []
    for hotel in hotels:
        # if counter>1: break
        counter+=1
        # print(hotel)
        hotel_name = hotel.find('h3').text.strip()
        
        hotel_link = 'https://www.tripadvisor.ru' + hotel.find('a')['href']
        print(f'Достопримечательность: {hotel_name}\nСсылка: {hotel_link}\n')
        time.sleep(random.randint(10, 20)) # Задержка между запросами
        dict_dostoprim = scrape_hotel_info(hotel_link)

        dict_dostoprim['Достопримечательность'] = hotel_name
        dostoprim_list.append(dict_dostoprim)
        
    print('counter: ', counter)
    return dostoprim_list
    

In [5]:
# hotel_list1 = scrape_hotels(base_url)

In [55]:
# hotel_list1

NameError: name 'hotel_list1' is not defined

In [93]:
# Переменная для хранения URL страницы
dostoprim_list = []
current_url = base_url
c = 0
# Основной цикл для скрапинга данных со всех страниц
while current_url:
    c+=1
    dostoprim_list_i = scrape_hotels(current_url)
    
    # Заходим на следующую страницу
    time.sleep(random.randint(10, 20)) # Задержка между запросами
    response = requests.get(current_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Поиск ссылки на следующую страницу
    next_button = soup.find('a', {'class':'BrOJk u j z _F wSSLS tIqAi unMkR', 'aria-label':'Next page'})
    print(next_button)
    if c==31:
        print('second page')
        break
    if next_button and 'href' in next_button.attrs:
        current_url = 'https://www.tripadvisor.ru' + next_button['href']
        print(current_url)
    else:
        current_url = None
    
    dostoprim_list.append(dostoprim_list_i)
    time.sleep(random.randint(3, 15)) # Задержка между запросами 

Достопримечательность: 1. Viasea Akvaryum
Ссылка: https://www.tripadvisor.ru/Attraction_Review-g293974-d10051500-Reviews-Viasea_Akvaryum-Istanbul.html

{'Адрес': 'Balıkçılar Sokak U Blok 20 D:226 Viaport Marina, Стамбул 34940 Турция', 'Название достопримечательности': 'Viasea Akvaryum', 'Общая оценка': '3.5', 'Кол-во отзывов': '75 отзывов', 'Тип достопримечательности': 'Парки развлечений и\xa0тематические парки'}
Достопримечательность: 2. Faruk Yalçın Zoo
Ссылка: https://www.tripadvisor.ru/Attraction_Review-g293974-d548913-Reviews-Faruk_YalcIn_Zoo-Istanbul.html

{'Адрес': 'Tuzla Cad. No: 297 Piri Reis Mh, Стамбул 41700 Турция', 'Название достопримечательности': 'Faruk Yalçın Zoo', 'Общая оценка': '3.5', 'Кол-во отзывов': '27 отзывов', 'Тип достопримечательности': 'Зоопарки'}
Достопримечательность: 3. Aslan Park
Ссылка: https://www.tripadvisor.ru/Attraction_Review-g293974-d19909985-Reviews-Aslan_Park-Istanbul.html

{'Адрес': 'Cami Mh., Balikcilar Sk. U Blok No:20 D:U/226 Tuzla, Стамбул 

In [94]:
# Создание DataFrame из списка списков словарей
# Объединяем все внутренние списки в один
flattened_dostoprim10 = [item for sublist in dostoprim_list for item in sublist]

# Создаем DataFrame
df_dostoprim10 = pd.DataFrame(flattened_dostoprim10)

In [ ]:
df_dostoprim_itog = pd.concat([df_dostoprim_itog, df_dostoprim10], ignore_index=True)

In [97]:
df_dostoprim_itog.to_csv('dostoprim_info.csv')